# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
df.head() #hi

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
df.dropna(inplace=True)
df['FL_Cabin'] = df['Cabin'].str.split('/').str[0]
df = df.drop(columns=['PassengerId', 'Name'])
df = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns)
dd = df['Transported']
df = df.drop(columns=['Transported'])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, dd, test_size = 0.20, random_state=0)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [6]:
bagging_reg.fit(X_train, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

- Evaluate your model

In [7]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
pred = bagging_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test, y_test))

MAE 0.25117455902406904
RMSE 0.38545408510026175
R2 score 0.4057005931180807


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [8]:
grid = {"max_leaf_nodes": [250, 500, 1000],
        "max_depth":[10,30,50]}

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
dt = DecisionTreeRegressor()

In [10]:
model = GridSearchCV(estimator = dt, param_grid = grid, cv=5, verbose=10)

- Run Grid Search

In [11]:
model.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 1/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.402 total time=   2.9s
[CV 2/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 2/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.363 total time=   2.6s
[CV 3/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 3/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.324 total time=   2.9s
[CV 4/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 4/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.345 total time=   3.0s
[CV 5/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 5/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.379 total time=   2.9s
[CV 1/5; 2/9] START max_depth=10, max_leaf_nodes=500............................
[CV 1/5; 2/9] END max_depth=10, ma

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [10, 30, 50],
                         'max_leaf_nodes': [250, 500, 1000]},
             verbose=10)

- Evaluate your model

In [12]:
model.best_params_

{'max_depth': 10, 'max_leaf_nodes': 250}

In [13]:
best_model = model.best_estimator_

In [15]:
pred = best_model.predict(X_test)

print(f"MAE: {mean_absolute_error(pred, y_test): .2f}")
print(f"RMSE: {mean_squared_error(pred, y_test, squared=False): .2f}")
print(f"R2 score:  {best_model.score(X_test, y_test): .2f}")

MAE:  0.27
RMSE:  0.41
R2 score:   0.34
